<a href="https://colab.research.google.com/github/mahmoudsoroor/ZTechnium-Internship-NLP-/blob/main/Key_Phrase_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install trafilatura
!pip install summa
!pip install git+https://github.com/smirnov-am/pytopicrank.git
!pip install git+https://github.com/LIAAD/yake
!pip install keyBERT
!pip install git+https://github.com/boudinfl/pke.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached urllib3-1.26.11-py2.py3-none-any.whl (139 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.11 which is incompatible.
google-api-core 1.31.6 requires six>=1.13.0, but you have six 1.11.0 which is incompatible.
allennlp 2.1.0 requires spacy<3.1,>=2.1.0, but you have spacy 3.4.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cl

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
!pip install -U spacy
!python -m spacy download en_core_web_sm
import spacy
nlp = spacy.load("en_core_web_sm")
! pip install flashtext

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2022-08-16 23:20:24.727177: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 7.8 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# T5
!pip install --quiet git+https://github.com/huggingface/transformers.git@5c00918681d6b4027701eb46cea8f795da0d4064
!pip install --quiet sentencepiece==0.1.95

# Allen NLP
!pip install allennlp==2.1.0 allennlp-models==2.1.0

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 2.2.2 requires transformers<5.0.0,>=4.6.0, but you have transformers 4.6.0.dev0 which is incompatible.
allennlp 2.1.0 requires spacy<3.1,>=2.1.0, but you have spacy 3.4.1 which is incompatible.
allennlp 2.1.0 requires transformers<4.4,>=4.1, but you have transformers 4.6.0.dev0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached spacy-3.0.8-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.8 MB)
  Using cached transformers-4.3.3-py3-none-any.whl (1.9 MB)
  Using cached typing_extensions-3.10.0.2-py3-none-any.whl (26 kB)
  Using cached thinc-8.0.17-cp37-cp37m-manylinux_2_17_x

In [ ]:
import trafilatura
import pandas as pd
import numpy as np
import unicodedata
from spacy import displacy
import nltk
from nltk.stem import PorterStemmer , SnowballStemmer ,LancasterStemmer , WordNetLemmatizer
from nltk.tokenize import sent_tokenize , word_tokenize
import re
import os
import string
import pke
from sklearn.feature_extraction.text import TfidfVectorizer

time: 5.62 s (started: 2022-08-17 00:27:36 +00:00)


read text

In [ ]:
path = "/content/Skill_BOK.txt"
with open(path,'rb') as f:
    text = f.read()
text = text.decode("utf-16")



time: 2.94 ms (started: 2022-08-16 23:45:38 +00:00)


In [ ]:
text

'Python (programming language)\r\nFrom Wikipedia, the free encyclopedia\r\nPython is an interpreted high-level general-purpose programming language. Its design philosophy emphasizes code readability with its use of significant indentation. Its language constructs as well as its object-oriented approach aim to help programmers write clear, logical code for small and large-scale projects.[30]\r\n\r\nPython is dynamically-typed and garbage-collected. It supports multiple programming paradigms, including structured (particularly, procedural), object-oriented and functional programming. It is often described as a "batteries included" language due to its comprehensive standard library.[31]\r\n\r\nGuido van Rossum began working on Python in the late 1980s, as a successor to the ABC programming language, and first released it in 1991 as Python 0.9.0.[32] Python 2.0 was released in 2000 and introduced new features, such as list comprehensions and a garbage collection system using reference coun

time: 4.53 ms (started: 2022-08-16 23:45:40 +00:00)


# preprocessing 

time: 1.23 ms (started: 2022-08-16 23:48:17 +00:00)


In [61]:
def remove_non_ascii(text):
    #Remove non-ASCII characters from list of tokenized words
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
def to_lowercase(text):
    return text.lower()

def remove_new_line(text):
    return re.sub ('[\t\n\r\f\v]','',text)

def remove_ar_word(text):
    return  re.sub('[^a-zA-z0-9@-_.% "]', '',text)  

def remove_whitespaces(text):
    return text.strip()

def remove_email(text):
  return re.sub('\S*@\S*\s?','',text)    


time: 3.21 ms (started: 2022-08-17 01:28:43 +00:00)


In [62]:
def clean_text( text):
    text = remove_non_ascii(text)
    text = to_lowercase(text)
    text = remove_new_line(text)
    text = remove_ar_word(text)
    text = remove_whitespaces(text)
    text = remove_email(text)

    return text


time: 1.43 ms (started: 2022-08-17 01:28:44 +00:00)


In [63]:
text = clean_text(text)

time: 5.51 ms (started: 2022-08-17 01:28:45 +00:00)


In [64]:
text

'python programming languagefrom wikipedia the free encyclopediapython is an interpreted highlevel generalpurpose programming language. its design philosophy emphasizes code readability with its use of significant indentation. its language constructs as well as its objectoriented approach aim to help programmers write clear logical code for small and largescale projects.python is dynamicallytyped and garbagecollected. it supports multiple programming paradigms including structured particularly procedural objectoriented and functional programming. it is often described as a "batteries included" language due to its comprehensive standard library.guido van rossum began working on python in the late 1980s as a successor to the abc programming language and first released it in 1991 as python 0.9.0. python 2.0 was released in 2000 and introduced new features such as list comprehensions and a garbage collection system using reference counting. python 3.0 was released in 2008 and was a major r

time: 4.71 ms (started: 2022-08-17 01:28:48 +00:00)


# keyphrase extraction model

In [65]:
extractor = pke.unsupervised.TfIdf()
extractor.load_document(input=text, language='en')
extractor.candidate_selection()
extractor.candidate_weighting()
keyphrases = extractor.get_n_best(n=20)
keyphrases


[('python', 1001.4954394835863),
 ('language', 87.11931812710414),
 ('released', 79.49772725037336),
 ('cpython', 71.79909090014934),
 ('compiler', 70.68907195617511),
 ('code', 67.67066041130894),
 ('statements', 65.91753633778606),
 ('division', 58.75647872652731),
 ('van rossum', 57.439272720119476),
 ('rossum', 57.439272720119476),
 ('standard library', 57.439272720119476),
 ('programming', 54.48517370742992),
 ('syntax', 53.4377909464033),
 ('monty', 50.259363630104545),
 ('string', 49.09527741609624),
 ('library', 47.69863635022402),
 ('java', 45.149761328589335),
 ('van', 44.75957271435023),
 ('indentation', 43.07945454008961),
 ('objectoriented', 43.07945454008961)]

time: 4.13 s (started: 2022-08-17 01:29:03 +00:00)


In [66]:
extractor = pke.unsupervised.TopicRank()
extractor.load_document(input=text, language='en')
extractor.candidate_selection()
extractor.candidate_weighting()
keyphrases = extractor.get_n_best(n=20)
keyphrases

[('python programming', 0.07058257969178723),
 ('highlevel generalpurpose programming language', 0.01631708064816525),
 ('code readability', 0.014057244422207944),
 ('programming', 0.012200102438469779),
 ('statements', 0.011740592178935107),
 ('generator expressions', 0.010537683795409054),
 ('operator', 0.009849012062475904),
 ('functional programming', 0.009825611193064483),
 ('examples', 0.00964255437095875),
 ('string format', 0.008277630921137178),
 ('variable names', 0.008253565774264837),
 ('dynamic typing', 0.008220376172146725),
 ('class', 0.00805118315347633),
 ('blocks', 0.00746951883031398),
 ('separate dynamicallyallocated object', 0.007316345030229541),
 ('division', 0.0069992564140486414),
 ('reference counting', 0.006819462820547959),
 ('releases', 0.006814006325307924),
 ('version', 0.006772675178368914),
 ('standard library', 0.006622060999404375)]

time: 5.93 s (started: 2022-08-17 01:29:12 +00:00)


In [67]:
extractor = pke.unsupervised.KPMiner()
extractor.load_document(input=text, language='en')
extractor.candidate_selection()
extractor.candidate_weighting()
keyphrases = extractor.get_n_best(n=20)
keyphrases

[('python', 1001.4954394835863),
 ('programming language', 215.39727270044804),
 ('van rossum', 172.31781816035843),
 ('new features', 107.69863635022402),
 ('guido van', 107.69863635022402),
 ('guido van rossum', 107.69863635022402),
 ('language', 87.11931812710414),
 ('released', 79.49772725037336),
 ('code', 67.67066041130894),
 ('python programming', 64.6191818101344),
 ('functional programming', 64.6191818101344),
 ('list comprehensions', 64.6191818101344),
 ('python community', 64.6191818101344),
 ('rossum', 57.439272720119476),
 ('programming', 54.48517370742992),
 ('van', 44.75957271435023),
 ('indentation', 43.07945454008961),
 ('objectoriented', 43.07945454008961),
 ('guido', 35.89954545007467),
 ('philosophy', 33.569679535762674)]

time: 4.26 s (started: 2022-08-17 01:29:22 +00:00)


In [68]:
extractor = pke.unsupervised.YAKE()
extractor.load_document(input=text, language='en')
extractor.candidate_selection()
extractor.candidate_weighting()
keyphrases = extractor.get_n_best(n=20)
keyphrases

[('python', 0.0011737094102197351),
 ('language', 0.009948596258835074),
 ('code', 0.011695080647365687),
 ('programming', 0.013387497639475157),
 ('statement', 0.01676209001711798),
 ('use', 0.01916297146247809),
 ('use python', 0.022225908398965585),
 ('programming language', 0.022241508041885148),
 ('python code', 0.025171211029221253),
 ('division', 0.025460830007335938),
 ('standard', 0.029935272580649982),
 ('like', 0.03152089030012309),
 ('operator', 0.0316363223183602),
 ('expressions', 0.0331713471310868),
 ('java', 0.03367868065907812),
 ('reference', 0.033843859489014055),
 ('monty python', 0.037127949367087895),
 ('including', 0.038885013125369586),
 ('python programming languagefrom', 0.03890375956652392),
 ('python syntax', 0.04049851906434444)]

time: 2.3 s (started: 2022-08-17 01:29:26 +00:00)


In [69]:
extractor = pke.unsupervised.TextRank()
extractor.load_document(input=text, language='en')
extractor.candidate_selection()
extractor.candidate_weighting()
keyphrases = extractor.get_n_best(n=20)
keyphrases

[('python language reference', 0.044447865101226725),
 ('standard python code', 0.0411142317968744),
 ('multiple stack levels.expressionssome python expressions',
  0.04010940732144015),
 ('active python core developers', 0.039680167530481),
 ('python programming', 0.03791521436319258),
 ('terminal type python', 0.037666164639209875),
 ('pythons extensive mathematics library', 0.03642701255800458),
 ('python interpreter.pythons developers', 0.0363693290269864),
 ('python code', 0.036293690471559614),
 ('pythons development team', 0.03608828523616144),
 ('python package index', 0.03457402459712868),
 ('python creator guido van rossum', 0.034526874842424594),
 ('monty python references', 0.03433790567226249),
 ('official python documentation', 0.03431149259246248),
 ('various python implementations', 0.03376347134136173),
 ('pascal.indentationmain article python syntax', 0.033610828640645095),
 ('python interpreter fivefold', 0.033237728648782994),
 ('pythons magic methods', 0.0332058449

time: 1.65 s (started: 2022-08-17 01:29:28 +00:00)


In [70]:
extractor = pke.unsupervised.SingleRank()
extractor.load_document(input=text, language='en')
extractor.candidate_selection()
extractor.candidate_weighting()
keyphrases = extractor.get_n_best(n=20)
keyphrases

[('python language reference', 0.07956960897256125),
 ('standard python code', 0.07053281062005162),
 ('python programming', 0.06738961458333526),
 ('active python core developers', 0.06696705163340926),
 ('multiple stack levels.expressionssome python expressions',
  0.06673877197456099),
 ('python creator guido van rossum', 0.06575323747977137),
 ('pythons extensive mathematics library', 0.06466915647826052),
 ('python code', 0.06431579169358949),
 ('python interpreter.pythons developers', 0.06352443562837964),
 ('pythons development team', 0.06348762754757697),
 ('monty python references', 0.0626285360660368),
 ('terminal type python', 0.06163945438613564),
 ('pascal.indentationmain article python syntax', 0.060677777042833725),
 ('various python implementations', 0.06064727756924518),
 ('justintime python compilers', 0.06041576542941749),
 ('python package index', 0.060313613296272946),
 ('official python documentation', 0.06030001478332986),
 ('article python syntax', 0.06003450442

time: 1.65 s (started: 2022-08-17 01:29:30 +00:00)


In [71]:
extractor = pke.unsupervised.TopicalPageRank()
extractor.load_document(input=text, language='en')
extractor.candidate_selection()
extractor.candidate_weighting()
keyphrases = extractor.get_n_best(n=20)
keyphrases

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  % sorted(inconsistent)


[('python language reference', 0.0794442203288089),
 ('standard python code', 0.06951378319529808),
 ('python programming', 0.06633163766589377),
 ('multiple stack levels.expressionssome python expressions',
  0.06605596538056846),
 ('active python core developers', 0.06588362830947381),
 ('python creator guido van rossum', 0.06365339226294477),
 ('python code', 0.06337106645472279),
 ('pythons development team', 0.0626613656222436),
 ('python interpreter.pythons developers', 0.06254045698882772),
 ('monty python references', 0.0618859535772327),
 ('terminal type python', 0.06097889885212213),
 ('various python implementations', 0.060125866928958556),
 ('python package index', 0.059365772495870925),
 ('official python documentation', 0.05919885463270882),
 ('pythonmost python implementations', 0.05903188366383124),
 ('pascal.indentationmain article python syntax', 0.05899545916816896),
 ('justintime python compilers', 0.058756877157174744),
 ('meaning python extensions', 0.058630555664

time: 3.85 s (started: 2022-08-17 01:29:31 +00:00)


In [72]:
extractor = pke.unsupervised.PositionRank()
extractor.load_document(input=text, language='en')
extractor.candidate_selection()
extractor.candidate_weighting()
keyphrases = extractor.get_n_best(n=20)
keyphrases

[('python programming', 0.12687679189262402),
 ('python language reference', 0.12256045067207202),
 ('standard python code', 0.10871310782008697),
 ('python code', 0.10213983500630747),
 ('python interpreter.pythons developers', 0.09715679313275238),
 ('pythons development team', 0.09707702583930955),
 ('monty python references', 0.09696072582107129),
 ('python design decisions', 0.09580348093809407),
 ('terminal type python', 0.0952404249202261),
 ('various python implementations', 0.09404086548926088),
 ('python use', 0.09394982330327048),
 ('justintime python compilers', 0.09389461256354872),
 ('article python syntax', 0.09366994435263418),
 ('meaning python extensions', 0.09365967924972977),
 ('official python documentation', 0.0935762337747104),
 ('python package index', 0.09346362523423192),
 ('pythonmost python implementations', 0.09339523109579115),
 ('python implementation', 0.09300640661476643),
 ('python syntax', 0.09288546150889504),
 ('python interpreter', 0.09269070895796

time: 2.01 s (started: 2022-08-17 01:29:35 +00:00)


In [73]:
extractor = pke.unsupervised.MultipartiteRank()
extractor.load_document(input=text, language='en')
extractor.candidate_selection()
extractor.candidate_weighting()
keyphrases = extractor.get_n_best(n=20)
keyphrases

[('python programming', 0.11064806124033925),
 ('python', 0.02941461682373529),
 ('highlevel generalpurpose programming language', 0.01582629674794723),
 ('code readability', 0.010817913709934004),
 ('statements', 0.007957348054833894),
 ('functional programming', 0.007558672694093203),
 ('generator expressions', 0.007333658181124318),
 ('programming', 0.006982198310666922),
 ('language', 0.006765429590324075),
 ('wikipedia', 0.006661392225524531),
 ('operator', 0.005994694165642569),
 ('variable names', 0.005986512794376232),
 ('ides sagemath', 0.005924564808361719),
 ('standard library', 0.005717280665383722),
 ('use', 0.005610929386251507),
 ('intermediate bytecode', 0.0054954734830149005),
 ('dynamic typing', 0.005481186397110435),
 ('design philosophy', 0.00537653386190841),
 ('reference counting', 0.00522992255127974),
 ('free encyclopediapython', 0.005112311554234695)]

time: 10.7 s (started: 2022-08-17 01:29:37 +00:00)
